In [87]:
import pandas as pd
# from crewai_tools import (
#     DirectoryReadTool,
#     FileReadTool,
#     SerperDevTool,
#     WebsiteSearchTool
# )
import os
from openpyxl import load_workbook
import math

In [90]:
SPREADSHEET_FILEPATH = "investments.xlsx"

In [ ]:
from pydantic import BaseModel
class InvestmentRequest(BaseModel):
    sheet_name: str
    distribution_dict: dict

def execute_investments(
    request: InvestmentRequest
) -> None:
    """
    Updates the investment values in the given sheet
    based on the distribution dictionary.
    Use the return value of the get_investment_distribution
    function as the parameter distribution_dict.
    """
    wb = load_workbook(SPREADSHEET_FILEPATH)
    ws = wb[request.sheet_name]
    for cols in list(ws.rows)[0]:
        if cols.value == "Investment":
            investment_letter = cols.column_letter
    for idx, tick_xls in enumerate(ws["A"][1:], start=1):
        current_investment = ws[investment_letter][idx].value or 0
        for tick_inv, quant in request.distribution_dict.items():
            if tick_xls.value == tick_inv and not isinstance(current_investment, str):
                # print(current_investment, quant)
                print(ws[investment_letter][idx].value)
                ws[investment_letter][idx].value = current_investment + quant
    wb.save(SPREADSHEET_FILEPATH)

In [104]:
def update_investment_distributions(spreadsheet_file_path) -> None:
    wb = load_workbook(spreadsheet_file_path)
    total_values = []
    for sheet in wb.sheetnames[1:]:
        df = pd.read_excel(spreadsheet_file_path, sheet_name=sheet)
        invest_sum = round(float(df.iloc[:-1, -1].sum()), 2)
        total_values.append(invest_sum)
        for i in range(1, len(wb["Distribution"]["A"])):
            if wb["Distribution"]["A"][i].value == sheet:
                print(wb["Distribution"]["A"][i].value)
                wb["Distribution"]["D"][i].value = invest_sum
                print(wb["Distribution"]["D"][i].value)
    wb.save(spreadsheet_file_path)

In [ ]:
def get_investment_distribution(sheet_name: str, total_value: int) -> dict:
    """
    Given a total investment value, this function returns the 
    monetary value for each investment based on each investment percentage.
    """
    df = pd.read_excel("investments.xlsx", sheet_name=sheet_name).fillna(0)
    def_col = df.columns[0]
    distribution = {}
    sum_to_be_invested = 0
    new_value = total_value
    while math.ceil(sum_to_be_invested) != total_value:
        for _, row in df.iterrows():
            if row["Target Percentage"] >= row["Current Percentage"] and \
                row["Target Percentage"] < 1.0:
                if distribution.get(row[def_col]) is None:
                    distribution[row[def_col]] = row["Target Percentage"] * new_value
                else:
                    distribution[row[def_col]] += row["Target Percentage"] * new_value
        sum_to_be_invested = sum(list(distribution.values()))
        new_value = total_value - sum_to_be_invested
    for key in distribution.keys():
        distribution[key] = round(distribution[key], 2)
    return distribution

In [ ]:
def get_tick_information_from_excel_file(filepath: str, sheet_name: str, code_identifier: str) -> str:
    """
    Read the excel file contianing the investment information, and return a row of the 
    dataframe in a LLM readable format, based on the provided code
    """
    df = pd.read_excel(filepath, sheet_name=sheet_name)
    row_filter = df.loc[df["Tick"] == code_identifier].to_dict(orient="list")
    return '\n'.join([f"{key}: {value[0]}" for key, value in row_filter.items()])